In [143]:
import json
from neo4j import GraphDatabase
from typing import List

class KnowledgeGraphIngestor:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def setup_vector_index(self):
        query = """
        CREATE VECTOR INDEX `issue-embeddings`
        FOR (i:Issue) ON (i.embedding)
        OPTIONS {
            indexConfig: {
                `vector.dimensions`: 384,
                `vector.similarity_function`: 'cosine'
            }
        }
        """
        with self.driver.session() as session:
            session.run(query)

    def create_nodes(self, data):
        with self.driver.session() as session:
            for product in data["products"]:
                session.run("MERGE (p:Product {id: $id, name: $name, alias: $alias})", **product)

            for sub in data["subtopics"]:
                session.run("""
                    MERGE (s:SubTopic {
                        id: $id,
                        name: $name,
                        alias: $alias,
                        Constraints: $Constraints,
                        Access: $Access,
                        `Common Issues`: $Common_Issues
                    })
                """, **sub)

            for issue in data["issues"]:
                issue["embedding"] = []  # Placeholder; will be set later
                session.run("""
                    MERGE (i:Issue {id: $id, description: $description, keywords: $keywords, 
                                    frequency: $frequency, severity: $severity})
                """, **issue)

            for user in data["users"]:
                session.run("MERGE (u:User {id: $id, name: $name, email: $email, location: $location, band: $band, team: $team})", **user)

            for device in data["devices"]:
                session.run("""
                    MERGE (d:Device {id: $id, os: $os, lastupdate: $lastupdate, 
                                     os_version: $os_version, ram: $ram, 
                                     storage: $storage, model: $model, issued_on: $issued_on, 
                                     pending_updates: $pending_updates})
                """, **device)

            for ticket in data["tickets"]:
                session.run("""
                    MERGE (t:Ticket {
                        id: $id,
                        query: $ticket_query,
                        category: $category,
                        solved_by: $solved_by,
                        timestamp: $timestamp
                    })
                """, 
                id=ticket["id"],
                ticket_query=ticket["query"],
                category=ticket["category"],
                solved_by=ticket["solved_by"],
                timestamp=ticket["timestamp"]
                )


            for article in data["articles"]:
                session.run("""
                    MERGE (a:Article {id: $id, title: $title, content: $content, 
                                      keywords: $keywords, created_at: $created_at, updated_at: $updated_at})
                """, **article)

            for outage in data["outages"]:
                session.run("""
                    MERGE (o:Outage {id: $id, title: $title, description: $description, 
                                     start_time: $start_time, end_time: $end_time, 
                                     affected_services: $affected_services, status: $status, 
                                     impact: $impact, location: $location, 
                                     expected_resolution: $expected_resolution})
                """, **outage)

    def create_relationships(self, data):
        with self.driver.session() as session:
            def link(from_id, to_id, label_from, label_to, rel):
                session.run(f"""
                    MATCH (a:{label_from} {{id: $from_id}}), (b:{label_to} {{id: $to_id}})
                    MERGE (a)-[:{rel}]->(b)
                """, from_id=from_id, to_id=to_id)


            for rel in data["product-subtopic-relationships"]:
                link(rel["from"], rel["to"], "Product", "SubTopic", rel["relation"])

            for rel in data.get("user-device-relationships", []):
                link(rel["from"], rel["to"], "User", "Device", rel["relation"])

            for rel in data.get("user-ticket-relationships", []):
                link(rel["from"], rel["to"], "User", "Ticket", rel["relation"])

            for rel in data.get("ticket-issue-relationships", []):
                link(rel["from"], rel["to"], "Ticket", "Issue", rel["relation"])

            for rel in data.get("article-issue-relationships", []):
                link(rel["from"], rel["to"], "Issue", "Article", rel["relation"])

            for rel in data.get("product-outage-relationships", []):
                link(rel["from"], rel["to"], "Product", "Outage", rel["relation"])

            for rel in data.get("subtopic-issue-relationships", []):  # If this exists
                link(rel["from"], rel["to"], "SubTopic", "Issue", rel["relation"])

    def set_embeddings(self, embedding_fn):
        """
        embedding_fn: function that takes issue description and returns embedding (List[float])
        """
        with self.driver.session() as session:
            issues = session.run("MATCH (i:Issue) RETURN i.id AS id, i.description AS description")
            for record in issues:
                id_ = record["id"]
                desc = record["description"]
                embedding = embedding_fn(desc)
                session.run("""
                    MATCH (i:Issue {id: $id})
                    CALL db.create.setNodeVectorProperty(i, 'embedding', $embedding)
                """, id=id_, embedding=embedding)


    def delete_all_data(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
        #deleting indexes
        self.delete_indexes()
        print("All data deleted.")

    def delete_indexes(self):
        with self.driver.session() as session:
            indexes = session.run("SHOW INDEXES")
            indexes_to_drop = [record["name"] for record in indexes]
            print("Indexes to drop:", indexes_to_drop)
            for index in indexes_to_drop:
                try:
                    session.run(f"DROP INDEX `{index}`")
                except Exception as e:
                    print(f"Failed to drop index {index}: {e}")



In [133]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

def embed(text):
    model_path =  "/Users/abhishekbairagi/Desktop/experiments/devcon/sent-transformer/all-MiniLM-L6-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    # model = SentenceTransformer(model_path)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embedding[0]


In [157]:
from google import genai
client = genai.Client(api_key="")
import json

def generate_text(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
    )
    # print(response.text)
    return response.text[7:-3]  if '```json' in response.text  else  response.text

# generate_text('hi')

In [145]:
#reading data
import json
with open("/Users/abhishekbairagi/Desktop/experiments/devcon/data generation/synthetic_data.json", "r") as f:
    data = json.load(f)


In [146]:
len(data['articles'])

28

In [147]:
# Example usage
kg_password = "strongpass123"
uri = "bolt://localhost:7687"
user = "neo4j"
kg = KnowledgeGraphIngestor(uri,user ,kg_password)
kg.delete_all_data()  # Clear existing data if needed
kg.setup_vector_index()
kg.create_nodes(data)
kg.create_relationships(data)


kg.set_embeddings(embed)
# kg.close()



Indexes to drop: ['issue-embeddings']
All data deleted.


In [148]:
def count_article_nodes():
    with driver.session() as session:
        result = session.run("MATCH (a:Article) RETURN count(a) AS article_count")
        article_count = result.single()["article_count"]
    print(f"Total Article Nodes: {article_count}")
    return article_count

count_article_nodes()

Total Article Nodes: 28


28

In [150]:
def get_articles_for_issue(issue_id):
    """
    Fetch all articles associated with a specific issue ID based on the 'related_to' relationship.
    """
    with kg.driver.session() as session:
        query = """
        MATCH (i:Issue {id: $issue_id})<-[:related_to]-(a:Article)
        RETURN i.id AS issue_id, a.id AS article_id, a.title AS title, a.content AS content
        """
        result = session.run(query, issue_id=issue_id)
        articles = [record.data() for record in result]
    return articles

# Example usage
articles_for_issue3 = get_articles_for_issue("issue3")
for article in articles_for_issue3:
    print(f"Article ID: {article['article_id']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['content']}")
    print("---")

In [155]:
def search_issues(query_text, product_name, subtopic_name=None, threshold=0.5, top_k=5):
    """
    Search relevant issues by semantic similarity and product/subtopic context,
    and return associated product, subtopic, and articles.
    """
    driver = GraphDatabase.driver(uri, auth=(user, kg_password))
    query_embedding = embed(query_text)

    cypher = """
    MATCH (p:Product)
    WHERE p.name = $product_name OR $product_name IN p.alias

    OPTIONAL MATCH (p)-[:has_subtopic]->(s:SubTopic)
    WHERE $subtopic_name IS NULL OR s.name = $subtopic_name OR $subtopic_name IN s.alias

    OPTIONAL MATCH (s)-[:has_issue]->(i:Issue)
    WHERE i.embedding IS NOT NULL

    WITH p, s, i, vector.similarity.cosine(i.embedding, $query_embedding) AS score
    WHERE score > $threshold

    OPTIONAL MATCH (i)-[:solved_by]->(a:Article)

    RETURN 
        i.id AS issue_id,
        i.description AS description,
        score,
        p.name AS product,
        s.name AS subtopic,
        collect(DISTINCT {
            id: a.id,
            title: a.title,
            content: a.content
        }) AS related_articles
    ORDER BY score DESC
    LIMIT $top_k
    """

    with driver.session() as session:
        result = session.run(
            cypher,
            product_name=product_name,
            subtopic_name=subtopic_name,
            query_embedding=query_embedding,
            threshold=threshold,
            top_k=top_k
        )
        return [record.data() for record in result]


In [156]:
results = search_issues(
    query_text="can't schedule a meeting",
    product_name="Webex",
    subtopic_name="Meeting Scheduling",
    threshold=0.6,
    top_k=3
)

# results = search_issues(
#     query_text="bot commands not working",
#     product_name="Slack",
#     subtopic_name="Bot Configurations",
#     threshold=0.55,
#     top_k=3
# )

# results = search_issues(
#     query_text="emails not syncing properly",
#     product_name="Outlook",
#     subtopic_name="Notifications",
#     threshold=0.6,
#     top_k=4
# )
for res in results:
    print(f"Issue ID: {res['issue_id']}")
    print(f"Description: {res['description']}")
    print(f"Product: {res['product']}")
    print(f"Subtopic: {res.get('subtopic', 'N/A')}")
    print(f"Score: {res['score']:.4f}")

    articles = res.get('related_articles', [])
    if articles:
        print("Related Articles:")
        for art in articles:
            print(f" - {art['title']} (ID: {art['id']})")
    else:
        print("Related Articles: None")
    print("---")


Issue ID: issue1
Description: Can't schedule meeting
Product: Webex
Subtopic: Meeting Scheduling
Score: 0.9948
Related Articles:
 - Troubleshooting Issues with Meeting Scheduling (ID: article1)
---
Issue ID: issue2
Description: Meeting not showing on calendar
Product: Webex
Subtopic: Meeting Scheduling
Score: 0.8269
Related Articles:
 - Why Meetings May Not Appear in Your Calendar (ID: article2)
---


In [160]:
with open("/Users/abhishekbairagi/Desktop/experiments/devcon/data generation/synthetic_data.json", "r") as f:
    data = json.load(f)
# Filter articles with the specified IDs
article_ids_to_read = ["article2", "article27", "article28"]
filtered_articles = [article for article in data["articles"] if article["id"] in article_ids_to_read]

# Print the filtered articles
for article in filtered_articles:
    print(f"ID: {article['id']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['content']}")
    print("---")


context = ""
for article in filtered_articles:
    context += f"\nDoc Id: {article['id']}\nTitle: {article['title']}\nContent: {article['content']}\n\n"



ID: article2
Title: Why Meetings May Not Appear in Your Calendar
Content: When a scheduled meeting fails to show up in your calendar, it can disrupt productivity and create confusion. The root of the issue often lies in sync problems between your calendar app and the meeting platform. Start by checking whether your calendar is set to auto-refresh or sync in real-time. If not, force a manual sync.

Ensure that you are logged into the correct account, especially if you use multiple email addresses. Sometimes meetings are scheduled in one calendar but are viewed in another. Also, check for any filters or views that may hide certain event types. For example, in Outlook or Google Calendar, make sure that your calendar list includes the one where the meeting was created.

Next, verify that you received a calendar invite and accepted it. If not accepted, it may not display depending on your settings. In corporate environments, calendar display issues can stem from permission misconfigurations

In [161]:
query = """I need help with webex audio?"""
rag_prompt = f"""Given query and certain documents genrate the answer from all the documents. Include \n wherever necessary.
Also mention the document id you are using to generate the answer.

Documents: 
{context}


Query:
{query}


"""
response = generate_text(rag_prompt)
print("Generated Response:")
print(response)

Generated Response:
Here's some guidance to troubleshoot Webex audio issues based on the provided documents:

*   **Check Webex audio settings:** Navigate to 'Audio Settings' in Webex to test your speaker and microphone and ensure the correct devices are selected. During calls, use the in-call device selector to switch output/input devices as needed (Doc Id: article28).
*   **Verify system permissions:** Ensure Webex has permission to access the microphone and speaker in your operating system settings (Doc Id: article28). On macOS, go to 'System Settings' > 'Privacy & Security' > 'Microphone' to ensure Webex is granted access (Doc Id: article27).
*   **macOS Sound Settings:** If you are using macOS, go to 'System Settings' > 'Sound' and confirm the correct output device is selected and not muted. Also, check 'Input' settings to ensure your microphone is correctly recognized (Doc Id: article27).
*   **Bluetooth devices:** If using Bluetooth devices, confirm that they are connected and s

In [ ]:
def search_issues2(query_text, product_name, subtopic_name=None, threshold=0.5, top_k=5):
    """
    Semantic issue search with KG context and triple extraction for RAG grounding.
    Returns scored issues and relevant KG triples.
    """
    driver = GraphDatabase.driver(uri, auth=(user, kg_password))
    query_embedding = embed(query_text)

    cypher = """
    MATCH (p:Product)
    WHERE p.name = $product_name OR $product_name IN p.alias

    OPTIONAL MATCH (p)-[:has_subtopic]->(s:SubTopic)
    WHERE $subtopic_name IS NULL OR s.name = $subtopic_name OR $subtopic_name IN s.alias

    OPTIONAL MATCH (s)-[:has_issue]->(i:Issue)
    WHERE i.embedding IS NOT NULL

    WITH p, s, i, vector.similarity.cosine(i.embedding, $query_embedding) AS score
    WHERE score > $threshold

    OPTIONAL MATCH (i)-[:solved_by]->(a:Article)
    OPTIONAL MATCH (i)<-[r1]-(:User)
    OPTIONAL MATCH (i)-[r2]->(n2)
    OPTIONAL MATCH (i)<-[r3]-(n3)

    WITH i, a, score, p, s,
         collect(DISTINCT {
             id: a.id, 
             title: a.title
         }) AS related_articles,
         collect(DISTINCT [startNode(r2).id, type(r2), endNode(r2).id]) + 
         collect(DISTINCT [startNode(r3).id, type(r3), endNode(r3).id]) AS kg_triples

    RETURN 
        i.id AS issue_id,
        i.description AS description,
        score,
        p.name AS product,
        s.name AS subtopic,
        related_articles,
        kg_triples
    ORDER BY score DESC
    LIMIT $top_k
    """

    with driver.session() as session:
        result = session.run(
            cypher,
            product_name=product_name,
            subtopic_name=subtopic_name,
            query_embedding=query_embedding,
            threshold=threshold,
            top_k=top_k
        )
        return [record.data() for record in result]


results = search_issues2(
    query_text="I need help with webex audio?",
    product_name="Webex",
    subtopic_name="Audio Issues"
)

if results:
    top = results[0]
    
    # Base context from articles (RAG)
    context_articles = top["related_articles"]

    # Knowledge triples (KG grounding)
    kg_triples = top["kg_triples"]

    # Debug print or feed to LLM
    print("Articles Context:")
    for article in context_articles:
        print(f"Article Title: {article['title']}")
        print(f"Article Content: {article['content']}")
        print("---")

    print("KG Triples:")
    for triple in kg_triples:
        start_node = triple[0]
        relation = triple[1]
        end_node = triple[2]
        print(f"Start Node: {start_node}, Relation: {relation}, End Node: {end_node}")
        print(f"Issue Description: {top['description']}")
        print(f"Subtopic Name: {top.get('subtopic', 'N/A')}")
        print(f"Product Name: {top.get('product', 'N/A')}")
        for article in context_articles:
            print(f"Related Article Title: {article['title']}")
        print("---")


Articles Context:
Article Title: Resolving Audio Failures in Webex Calls Due to App Settings
Article Content: Webex may experience audio failures during calls if its internal audio settings are misconfigured or if there's a conflict with system audio. First, open Webex and navigate to 'Audio Settings' before joining a call to test your speaker and microphone. Ensure the correct devices are selected. During calls, use the in-call device selector to switch output/input devices as needed. Check whether Webex has permission to access microphone and speaker resources in your operating system settings. If using headphones or Bluetooth devices, connect them before launching Webex. In some cases, corrupted Webex cache files can interfere with audio—clearing the app cache or reinstalling can help. Lastly, network issues or firewalls may block audio ports; verify that the required UDP ports (e.g., 5004, 33434-33598) are not blocked. Contact support if issues continue despite following these step

In [168]:
kg_triplets_context = "\n\n".join(
    f"Start Node: {triple[0]}, Relation: {triple[1]}, End Node: {triple[2]}\n"
    f"Issue Description: {top['description']}\n"
    f"Subtopic Name: {top.get('subtopic', 'N/A')}\n"
    f"Product Name: {top.get('product', 'N/A')}\n"
    f"Related Article Title: {article['title']}"
    for triple in kg_triples
    for article in top["related_articles"]
)
print("KG Triplets Context:")
print(kg_triplets_context)

KG Triplets Context:
Start Node: issue3, Relation: solved_by, End Node: article28
Issue Description: No audio during call
Subtopic Name: Audio Issues
Product Name: Webex
Related Article Title: Resolving Audio Failures in Webex Calls Due to App Settings

Start Node: issue3, Relation: solved_by, End Node: article27
Issue Description: No audio during call
Subtopic Name: Audio Issues
Product Name: Webex
Related Article Title: Resolving Audio Failures in Webex Calls Due to App Settings

Start Node: issue3, Relation: solved_by, End Node: article3
Issue Description: No audio during call
Subtopic Name: Audio Issues
Product Name: Webex
Related Article Title: Resolving Audio Failures in Webex Calls Due to App Settings

Start Node: sub2, Relation: has_issue, End Node: issue3
Issue Description: No audio during call
Subtopic Name: Audio Issues
Product Name: Webex
Related Article Title: Resolving Audio Failures in Webex Calls Due to App Settings

Start Node: INC792834, Relation: of_issue, End Node: 

In [175]:
def get_relevant_issues_with_articles(product_name, subtopic_name, query_text, threshold=0.5, top_k=5):
    """
    Fetch top semantically matching issues for a given product and subtopic from the knowledge graph,
    including articles related to the issues via the 'solved_by' relationship.
    """
    driver = GraphDatabase.driver(uri, auth=(user, kg_password))
    query_embedding = embed(query_text)

    cypher = """
    MATCH (p:Product)
    WHERE toLower(p.name) = toLower($product_name) OR any(alias IN p.alias WHERE toLower(alias) = toLower($product_name))

    OPTIONAL MATCH (p)-[:has_subtopic]->(s:SubTopic)
    WHERE $subtopic_name IS NULL OR any(word IN split(toLower($subtopic_name), " ") WHERE word IN split(toLower(s.name), " ") OR any(alias IN s.alias WHERE toLower(alias) = word))

    OPTIONAL MATCH (s)-[:has_issue]->(i:Issue)
    WHERE i.embedding IS NOT NULL

    OPTIONAL MATCH (i)-[:solved_by]->(a:Article)

    WITH p, s, i, a, vector.similarity.cosine(i.embedding, $query_embedding) AS score
    WHERE score > $threshold

    RETURN 
        i.id AS issue_id,
        i.description AS description,
        score,
        p.name AS product,
        s.name AS subtopic,
        collect(DISTINCT {id: a.id, title: a.title, content: a.content}) AS related_articles
    ORDER BY score DESC
    LIMIT $top_k
    """

    with driver.session() as session:
        result = session.run(
            cypher,
            product_name=product_name,
            subtopic_name=subtopic_name,
            query_embedding=query_embedding,
            threshold=threshold,
            top_k=top_k
        )
        return [record.data() for record in result]

# Example usage
relevant_issues_with_articles = get_relevant_issues_with_articles(
    product_name="webex",
    subtopic_name="Audio",
    query_text="I can't hear sound during my Webex call",
    threshold=0.6,
    top_k=5
)

kg_context = ""
for issue in relevant_issues_with_articles:
    print(f"Issue ID: {issue['issue_id']}")
    print(f"Description: {issue['description']}")
    print(f"Product: {issue['product']}")
    print(f"Subtopic: {issue['subtopic']}")
    print(f"Score: {issue['score']:.4f}")
    print("Related Articles:")
    for article in issue['related_articles']:
        print(f" - {article['title']} (ID: {article['id']})")
    print("---")
    
    # Add to kg_context
    kg_context += f"Issue ID: {issue['issue_id']}\n"
    kg_context += f"Description: {issue['description']}\n"
    kg_context += f"Product: {issue['product']}\n"
    kg_context += f"Subtopic: {issue['subtopic']}\n"
    kg_context += f"Score: {issue['score']:.4f}\n"
    kg_context += "Related Articles:\n"
    for article in issue['related_articles']:
        kg_context += f" - {article['title']} (ID: {article['id']})\n"
    kg_context += "---\n"





Issue ID: issue3
Description: No audio during call
Product: Webex
Subtopic: Audio Issues
Score: 0.8391
Related Articles:
 - Resolving Audio Failures in Webex Calls Due to App Settings (ID: article28)
 - Fixing Audio Issues During Webex Calls on macOS (ID: article27)
 - Troubleshooting No Sound During Calls in Windows (ID: article3)
---
Issue ID: issue4
Description: Microphone not detected
Product: Webex
Subtopic: Audio Issues
Score: 0.7192
Related Articles:
 - Fixing Microphone Detection Problems (ID: article4)
---


In [180]:
kg_prompt = """Given the query and the context from articles and knowledge graph triplets related to query, generate a comprehensive response to user query. 
You can either choose to answer or ask a followup question to get more details. 
Use the context from articles and knowledge graph triplets to generate the response.
Query:
{query}

User Profile: 
User OS: Windows

Knowledge Graph Triplets:
{kg_triples}

Context from Articles:
{context_articles}
Response:
"""
kg_prompt = kg_prompt.format(
    query="I hear no sound on my Webex call",
    kg_triples=f"Related Issues from Knowledge Graph - {kg_context}",
    context_articles=json.dumps(context_articles, indent=2)
)


response = generate_text(kg_prompt)
print(response)



Since you're not hearing any sound on your Webex call and you're using Windows, here's a troubleshooting approach based on common causes and solutions:

1.  **Check Webex Audio Settings:**

    *   Open Webex and go to 'Audio Settings' *before* joining a call.
    *   Test your speaker and microphone to ensure they are working within the app.
    *   Make sure the correct speaker/output device is selected.
    *   During a call, use the in-call device selector to switch between available output devices if necessary (article28).

2.  **Operating System Permissions:**

    *   Verify that Webex has permission to access your microphone and speaker within your Windows system settings (article28).

3.  **Headphones/Bluetooth Devices:**

    *   If you're using headphones or Bluetooth devices, connect them *before* launching Webex (article28).

4.  **Webex App Cache:**

    *   Corrupted Webex cache files can sometimes cause audio problems. Try clearing the app cache or reinstalling Webex (a

In [ ]:
query = """I am unable to hear sound during my Webex call. Can you help me with this issue?"""
context_articles_rag = '''
Article1:
"title": "Troubleshooting Issues with Meeting Scheduling",
"content": "If you're experiencing problems when trying to schedule a meeting, there are several possible causes and corresponding solutions. First, check your calendar permissions. If your calendar is shared or connected to another service (like Outlook or Google Calendar), ensure that you have the proper permissions to create events. In some cases, meeting scheduling may fail due to sync delays or outdated calendar tokens.\n\nAnother common issue involves time zone mismatches between participants. Always confirm that all participants' calendars are set to the correct time zone. Additionally, browser extensions or security software can sometimes block the scripts required to render the scheduling interface.\n\nClear your browser's cache and cookies, or try scheduling the meeting in a different browser or in incognito/private mode. If you're using a third-party scheduling tool, ensure it’s properly integrated with your primary calendar service.\n\nAlso check if the service’s backend (such as the scheduling API or cloud calendar service) is currently facing any outages. You can usually find this information on the service status page. If the issue persists, reach out to your IT admin to verify there are no policy restrictions on your account preventing scheduling actions.",

Article2:


'''

In [ ]:
rag_case1_prompt = f'''
You are a customer support agent for a tech company. Your job is to answer technical questions of user.

Instructions:
- Make sure to use the context provided to answer the question.
- Assure that no information is lost from the context, giving complete information.
- If you are not sure about the answer, ask a followup question to get more details.
- If you are sure about the answer, provide a comprehensive response.
User Query:
{query}
Context from Articles:
{context_articles}
Output:
'''

In [ ]:
import requests
import json

# Elasticsearch URL
es_url = "http://localhost:9200/articles"

# Define the index mapping
index_mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "title": {"type": "text"},
            "content": {"type": "text"},
            "keywords": {"type": "keyword"},
            "created_at": {"type": "date"},
            "updated_at": {"type": "date"},
            "embedding": {
                "type": "dense_vector",
                "dims": 384
            }
        }
    }
}

# Create the index
response = requests.put(es_url, headers={"Content-Type": "application/json"}, data=json.dumps(index_mapping))
if response.status_code == 200:
    print("Index created successfully.")
else:
    print(f"Failed to create index: {response.text}")

Failed to create index: {"error":{"root_cause":[{"type":"resource_already_exists_exception","reason":"index [articles/SMlIVoxbSimPEtsVp0dxJA] already exists","index_uuid":"SMlIVoxbSimPEtsVp0dxJA","index":"articles"}],"type":"resource_already_exists_exception","reason":"index [articles/SMlIVoxbSimPEtsVp0dxJA] already exists","index_uuid":"SMlIVoxbSimPEtsVp0dxJA","index":"articles"},"status":400}
{'id': 'article1', 'title': 'Troubleshooting Issues with Meeting Scheduling', 'content': "If you're experiencing problems when trying to schedule a meeting, there are several possible causes and corresponding solutions. First, check your calendar permissions. If your calendar is shared or connected to another service (like Outlook or Google Calendar), ensure that you have the proper permissions to create events. In some cases, meeting scheduling may fail due to sync delays or outdated calendar tokens.\n\nAnother common issue involves time zone mismatches between participants. Always confirm that

In [ ]:
# Read articles from the file
with open("/Users/abhishekbairagi/Desktop/experiments/devcon/data generation/synthetic_data.json", "r") as f:
    data = json.load(f)

# Ingest articles into Elasticsearch
for article in data["articles"]:
    # Convert numpy.float32 to standard Python float
    article["embedding"] = [float(value) for value in embed(article["content"])]  # Assuming the embed function returns a list of floats

    # Ingest the article
    print(article)
    ingest_response = requests.post(f"{es_url}/_doc/{article['id']}", headers={"Content-Type": "application/json"}, data=json.dumps(article))   
    if ingest_response.status_code == 201:
        print(f"Article {article['id']} ingested successfully.")
    else:
        print(f"Failed to ingest article {article['id']}: {ingest_response.text}")

In [199]:
import requests
import json

def semantic_search(query, es_url, index_name, embed_fn, top_k=5):
    """
    Perform semantic search on Elasticsearch index using query embedding.

    Args:
        query (str): The search query.
        es_url (str): Elasticsearch base URL.
        index_name (str): Name of the Elasticsearch index.
        embed_fn (function): Function to generate embedding for the query.
        top_k (int): Number of top results to return.

    Returns:
        list: List of top matching documents with their scores.
    """
    # Generate embedding for the query
    query_embedding = [float(value) for value in embed(query)]
    # Construct the search query for Elasticsearch
    search_query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match": {"content":query}},
                "script": {
                    "source": "_score*2 + cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": query_embedding}
                }
            }
        }
    }

    # Perform the search request
    response = requests.post(
        f"{es_url}/_search",
        headers={"Content-Type": "application/json"},
        data=json.dumps(search_query)
    )

    # Check for errors
    if response.status_code != 200:
        raise Exception(f"Failed to perform search: {response.text}")

    # Parse the response
    results = response.json()["hits"]["hits"]
    return [{"id": hit["_id"], "score": hit["_score"], "source": hit["_source"]} for hit in results]

# Example usage
query = "I am facing issue with webex meetings"
top_results = semantic_search(query, es_url, "articles", embed, top_k=5)
for result in top_results:
    print(f"ID: {result['id']} Score: {result['score']:.4f}")
    print(f"Title: {result['source']['title']}")
    print(f"Content: {result['source']['content']}")
    print("---")

ID: article28 Score: 12.2933
Title: Resolving Audio Failures in Webex Calls Due to App Settings
Content: Webex may experience audio failures during calls if its internal audio settings are misconfigured or if there's a conflict with system audio. First, open Webex and navigate to 'Audio Settings' before joining a call to test your speaker and microphone. Ensure the correct devices are selected. During calls, use the in-call device selector to switch output/input devices as needed. Check whether Webex has permission to access microphone and speaker resources in your operating system settings. If using headphones or Bluetooth devices, connect them before launching Webex. In some cases, corrupted Webex cache files can interfere with audio—clearing the app cache or reinstalling can help. Lastly, network issues or firewalls may block audio ports; verify that the required UDP ports (e.g., 5004, 33434-33598) are not blocked. Contact support if issues continue despite following these steps.
--